# MODS Archon FITS header lab

Lab notebook for deconstructing, fixing, and working with azcam/archon-generated FITS headers from the MODS instruments.



In [ ]:
import os
import numpy as np

# Set up matplotlib

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

%matplotlib inline

# astropy packages we need

from astropy.io import fits
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time

# other bits

import time

## Plot setup

In [ ]:
# Aspect ratio

aspect = 2.5

#
# Don't change these unless you really need to (we never have).  They make for good resolution
# and scale for insertion into LaTeX and Word documents
#
# fPage is the horizontal fraction of the page occupied by the figure, default 1.0
#
# scaleFac is the LaTeX includegraphics scaling in units of \textwidth, default 1.0
#

fPage = 1.0
scaleFac = 1.0

# Text width in inches - don't change, this is defined by the print layout for most portrait LaTeX templates

textWidth = 6.0 # inches

# Graphic dimensions depending on bitmap or vector format (draft vs production)

figFmt = 'png'
dpi = 600
plotWidth = dpi*fPage*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 8
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = fPage*textWidth # float(plotWidth)/float(dpi)
hInches = wInches/aspect  # float(plotHeight)/float(dpi)

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad']=f'{axisPad}'
plt.rcParams['ytick.major.pad']=f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## MODS CCD Image Basics

Read and sniff the headers of a MODS FITS file.  Format is multi-extension FITS with 5 extensions
 * PRIMARY - primary HDU with the full FITS header created by azcam including instrument and telescope when enabled.
 * Q1 - Image HDU for CCD quadrant 1
 * Q2 - Image HDU for CCD quadrant 2
 * Q3 - Image HDU for CCD quadrant 3
 * Q4 - Image HDU for CCD quadrant 4
 * CONPARS - BinTable HDU with the Archon controller status snapshot at start of exposure

### CCD Layout in detector coordinates

<img src="MODS_CCD_Readout.png" width="800">

A full frame is 8288x3088 pixels.  There are 32 columns of overscan in each quadrant, so the total number of pixels in each
image file is 8352x3088. The central reference pixel is (4144,1544) for converting to full detector coordinates.

The 4 quadrants are labeled Q1 through Q4 as shown.  Each is 4176x1544 pixels (4144 active + 32 overscan columns).  Quadrants 1 and 3
are readout toward the left corner amplifiers, quadrants 2 and 4 are readout toward the right corner amplifiers.  

A full-frame unbinned image is readout in about 20 seconds with a 0.5 second shutter delay before readout starts.

Subframe region-of-interest (ROI) readout modes support symmetical windows centered on the reference pixel, and are supported
for 1024x1024, 3088x3088, and 4196x3088 sizes, each with an additional 32 columns of overscan per quadrant.

## Open the image, and scan the header.

In [ ]:
inFile = "2025Dec11/mods2r.20251210.0012.fits"
hdu = fits.open(inFile)
hdu.info()

print("\nPrimary FITS Header:\n")

hdu[0].header

hdu.close()
#print("\nQ1 Image FITS Header:\n")
#
#hdu[1].header

## Quadrant Images

Analysis of coordinates and the bias overscan columns. 

Some questions:
 * what is the bias level?
 * how clean is the bias?
 * How many initial columns to skip?
 * Is the bias flat or is there a trend with row?

In [ ]:
#inFile = "2025Dec11/mods2r.20251210.0011.fits"
inFile = "2025Dec10/mods2r.20251210.0010.fits"

# open the file

hdu = fits.open(inFile)

# plotting options

showOverscan = False

# Q2 and Q4 are flipped along rows

flipRows = [False,True,False,True]

# bias column analysis

biasColOff = 2 # skip 2 columns at start of biassec
biasRowOff = 2 # skip 2 rows at start/end of biassec 

# reference pixels in x and y

xref = int(hdu[0].header["ref-pix1"])
yref = int(hdu[0].header["ref-pix2"])

print(f"Reference pixel: {xref},{yref}")

# Size of data section w/o column bias

ncImg = 2*(hdu[1].header['naxis1'] - hdu[1].header['ovrscan1'])
nrImg = 2*hdu[1].header['naxis2']

print(f"Output size: {ncImg}x{nrImg}")
fullImg = np.empty((nrImg,ncImg),dtype=np.float32)

# quadrant

for quad in [1,2,3,4]:
    
    t0 = time.perf_counter() # start performance timer
    
    quadData = hdu[quad].data.astype(np.float32) # convert to 32-bit float for processing
    
    nc = hdu[quad].header['naxis1'] # number of columns
    nr = hdu[quad].header['naxis2'] # number of rows
    ncbias = hdu[quad].header['ovrscan1'] # number of overscan columns
    print(f"\nImage Q{quad}:")
    print(f"  Format: {nc}x{nr}, {ncbias} overscan columns")
    print(f"  dataType: {quadData.dtype}")
    print(f"  BIASSEC={hdu[quad].header['biassec']}")
    print(f"  DATASEC={hdu[quad].header['datasec']}")
    print(f"  DETSEC={hdu[quad].header['detsec']}")
    print(f"  CCDSEC={hdu[quad].header['ccdsec']}")

    # extract bias overscan columns
    
    biasCols = quadData[biasRowOff:-biasRowOff,nc-ncbias+biasColOff:]
    #print(f"  biasCols shape: {biasCols.shape}")
    medBias = np.median(biasCols)
    sigBias = np.std(biasCols)
    minBias = np.min(biasCols)
    maxBias = np.max(biasCols)
    print(f"  Bias: Median={medBias:.2f} +/- {sigBias:.2f}, Min={minBias:.2f} Max={maxBias:.2f}")
    bias1d = np.median(biasCols,axis=1)
    print(f"        clipped median: {np.median(bias1d):.2f} +/- {np.std(bias1d):.2f}")

    # subtract bias from the data

    imgData = quadData[:,:nc-ncbias-1] - medBias
    # print(f"  imgData data type: {imgData.dtype}")
    
    if flipRows[quad-1]:
        imgData = np.flip(imgData,axis=1)
        print(f"  flipped along rows")
        
    t1 = time.perf_counter()
    biasTime = t1 - t0
    print(f"  debias time: {biasTime:.3f} sec")
    
    # basic image stats
    
    imgMed = np.median(imgData)
    imgStd = np.std(imgData)
    print(f"   Min: {np.min(imgData):.2f}")
    print(f"   Max: {np.max(imgData):.2f}")
    print(f"  Mean: {np.mean(imgData):.2f}, Median: {imgMed:.2f}")
    print(f"   Std: {imgStd:.2f}")

    t2 = time.perf_counter()
    statsTime = t2 - t1
    print(f"  stats time: {statsTime:.3f} sec")
    
    # plot bias subtracted image (or whole image)
    
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(wInches,hInches),dpi=dpi)
    fig.subplots_adjust(wspace=0.3, hspace=0.0)
    
    if showOverscan:
        ax1.imshow(quadData-medBias, cmap="gray",vmin=imgMed-medBias-imgStd,vmax=imgMed-medBias+imgStd)
    else:
        ax1.imshow(imgData, cmap="gray",vmin=imgMed-imgStd,vmax=imgMed+imgStd)
    ax1.tick_params('both',length=2,width=lwidth,which='major',direction='out',top='on',right='on')
    ax1.tick_params('both',length=1,width=lwidth,which='minor',direction='out',top='on',right='on')
    ax1.set_title(f"Q{quad} Bias-subtracted")
    
    # plot column bias vs. row pixel
    
    ax2.tick_params('both',length=4,width=lwidth,which='major',direction='in',top='on',right='on')
    ax2.tick_params('both',length=2,width=lwidth,which='minor',direction='in',top='on',right='on')
    ax2.plot(bias1d,'.',lw=0.5,ms=1,color='black')
    ax2.set_xlabel("Row pixel")
    ax2.set_ylabel("median bias")
    ax2.set_title(f"Q{quad} Overscan Bias")

    plt.show()

hdu.close()

## Debias and stitch the quadrants into as single image

Operations:
 * read in the image
   - compute full image size, create empty 32-bit float array
 * for each quadrant:
   - compute and subtract overscan bias
   - if Q2 or Q4, flip in rows
   - add to the full image
   

In [ ]:
inFile = "2025Dec10/mods2r.20251210.0010.fits"

# open the file
     
t0 = time.perf_counter() # start performance timer
    
hdu = fits.open(inFile)

# Q2 and Q4 are flipped along rows

flipRows = [False,True,False,True]

# bias column analysis

biasColOff = 2 # skip 2 columns at start of biassec
biasRowOff = 2 # skip 2 rows at start/end of biassec 

# reference pixels in x and y

xref = int(hdu[0].header["ref-pix1"])
yref = int(hdu[0].header["ref-pix2"])

print(f"Reference pixel: {xref},{yref}")

# Size of data section w/o column bias

ncQuad = hdu[1].header['naxis1'] - hdu[1].header['ovrscan1']
nrQuad = hdu[1].header['naxis2']

ncImg = 2*ncQuad
nrImg = 2*nrQuad

print(f"Output size: {ncImg}x{nrImg}")
fullImg = np.empty((nrImg,ncImg),dtype=np.float32)

# starting numpy pixels of each quadrant

startPix = [(0,0),(0,ncQuad),(nrQuad,0),(nrQuad,ncQuad)]

# quadrant

quadBias = []
quadBStd = []
for quad in [1,2,3,4]:
    quadData = hdu[quad].data.astype(np.float32) # convert to 32-bit float for processing
    nc = hdu[quad].header['naxis1'] # number of columns
    nr = hdu[quad].header['naxis2'] # number of rows
    ncbias = hdu[quad].header['ovrscan1'] # number of overscan columns

    # extract bias overscan columns
    
    biasCols = quadData[biasRowOff:-biasRowOff,nc-ncbias+biasColOff:]
    bias1d = np.median(biasCols,axis=1)
    medBias = np.median(bias1d)
    stdBias = np.std(bias1d)
    print(f"  Median overscan bias: {medBias:.2f} +/- {stdBias:.2f}")
    quadBias.append(medBias)
    quadBStd.append(stdBias)

    # subtract bias from the data

    imgData = quadData[0:,0:nc-ncbias] - medBias

    quadMed = np.median(imgData)
    quadStd = np.std(imgData)
    print(f"  Quad{quad}: median: {quadMed:.2f} +/- {quadStd:.2f}")
    
    # is this quadrant flipped? Unflip it

    if flipRows[quad-1]:
        imgData = np.flip(imgData,axis=1)
        print(f"  flipped along rows")
        
    # put the debiased quadrant into the full image array
    
    sr = startPix[quad-1][0]
    er = sr + nr
    sc = startPix[quad-1][1]
    ec = sc + nc - ncbias
    print(f"[{sr}:{er},{sc}:{ec}]")
    
    fullImg[sr:er,sc:ec] = imgData

# make an image header data unit for the merged image and insert it into the main HDU
# the full header from the primary HDU gets copied in

mergeHDU = fits.ImageHDU(data=fullImg,header=hdu[0].header,name="Merged")

for quad in [1,2,3,4]:
    mergeHDU.header[f"Q{quad}Bias"] = (quadBias[quad-1],f"Q{quad} median overscan bias [DN]")
    mergeHDU.header[f"Q{quad}Std"] = (quadBStd[quad-1],f"Q{quad} overscan bias stdev [DN]")

hdu.append(mergeHDU)

# write to a new file

rootName = os.path.splitext(inFile)[0]
newFITS = f"{rootName}_ot.fits"
hdu.writeto(newFITS,overwrite=True)

t1 = time.perf_counter()
print(f"Debias, merge, and save: {(t1-t0):.3f} sec")

hdu.close()

# quick stats and display 

imgMed = np.median(fullImg)
imgStd = np.std(fullImg)
print(f"Full image: median: {imgMed:.2f} +/- {imgStd:.2f} DN")

# plot bias subtracted image (or whole image)
    
fig,ax = plt.subplots(figsize=(wInches,hInches),dpi=dpi)

ax.imshow(fullImg, cmap="gray",vmin=imgMed-imgStd,vmax=imgMed+imgStd)
ax.tick_params('both',length=2,width=lwidth,which='major',direction='out',top='on',right='on')
ax.tick_params('both',length=1,width=lwidth,which='minor',direction='out',top='on',right='on')
ax.set_title(f"Merged, Bias-subtracted image")
    
plt.show()